<a href="https://colab.research.google.com/github/onepunchmen23/Agente-IA/blob/main/Meu_Projeto_para_a_Imers%C3%A3o_IA_Alura_%2B_Google_Gemini_Aula_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Instalar Framework ADK de agentes do Google ################################################
!pip -q install google-adk

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##########################################
# --- Agente 1: Buscador de Filmes/Séries e Cinemas/streamings --- #
##########################################

def agente_buscador(topico, data_de_hoje):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        description="Agente que busca notícias sobre diversos filmes lançados e melhores cinemas",
        tools=[google_search],
        instruction="""
        Você é um agente especializado em fornecer informações sobre a indústria cinematográfica.
        Sua tarefa é, utilizando a ferramenta de busca do Google (google_search), identificar os próximos lançamentos de filmes mais aguardados e também os melhores cinemas na região (Estado de São Paulo, Brasil) com base nas avaliações e comentários de clientes online.
        Para cada filme, liste o título, a data de lançamento prevista e um breve resumo da trama. Para cada cinema, liste o nome, o endereço e um resumo das avaliações dos clientes, destacando os pontos positivos e negativos mais comuns. Apresente os resultados de forma clara e organizada.
        Caso a informação seja uma série voce deve informar um resumo da série, informar os melhores canais de streamings com base nas avaliações dos clientes, utilizando as informações do Google (google_search) sem informações de cinemas e locais, apenas os melhores streamings.
            """
        # Inserir as instruções do Agente Buscador #################################################
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [ ]:
################################################
# --- Agente 2: Planejador --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
      Você é um agente planejador, sua função é ver o melhor dia e horário das sessões dos filmes informados e dos melhores cinemas e os melhores trajetos usando as informações do Google (google_search) e montar em forma de uma planilha completa.
      Caso a informação seja uma série voce deve informar os melhores canais de streamings com base nas avaliações dos clientes utilizando as informações do Google (google_search) e montar em forma de uma planilha completa.
        """,
        description="Agente que planeja e apresenta os melhores lançamentos de filmes com os melhores cinemas da região",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [ ]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de busca de melhores filmes/séries e cinemas/streamings para acompanhar em 2025 com 2 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o filme ou a série que deseja assistir: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
  print("Voce esqueceu de digitar o tópico!")
else:
  print(f"Maravilha! Vamos então criar o post sobre novidades em {topico}")

  lancamento = agente_buscador(topico, data_de_hoje)
  print("\n--- Resultado do agente 1 (Buscador) ---\n")
  display(to_markdown(lancamento))
  print("----------------------------------------------------------------")

  plano_de_post = agente_planejador(topico, lancamento)
  print("\n--- Resultado do agente 2 (Planejador) ---\n")
  display(to_markdown(plano_de_post))
  print("----------------------------------------------------------------")

🚀 Iniciando o Sistema de busca de melhores filmes/séries e cinemas/streamings para acompanhar em 2025 com 2 Agentes 🚀
